In [1]:
import pandas as pd
import pyrfume
import os

In [2]:
archives = pyrfume.list_archives()

# Skip over archives that are not for sole datasources
# Also skip Qian 2022 since not currently formatted to proper design scheme
skip_list = ['mordred', 'morgan', 'molecules', 'embedding', 'prediction_targets', 'tools', 'qian_2022']
archives = [arc for arc in archives if arc not in skip_list]

In [48]:
# pd.DataFrame(archives).to_csv(os.path.join('..', 'static', 'archive_list.csv'), header=None, index=None)

In [53]:
data_dict = {}
for arc in archives:
    num_molecules, has_behavior, has_physics = 0, False, False
    
    processed = pyrfume.load_manifest(arc)['processed']
    
    if 'molecules.csv' in processed:
        molecules = pyrfume.load_data(f'{arc}/molecules.csv')
        num_molecules = molecules.shape[0]
    
    for key in processed.keys():
        if 'behavior' in key:
            has_behavior = True  
        if 'physics' in key:
            has_physics = True
    
    data_dict[arc] = {'# Molecules': num_molecules, 'Behavior': has_behavior, 'Physics': has_physics}

In [55]:
tmp = pd.DataFrame.from_dict(data_dict, orient='index')
tmp = tmp.sort_values('# Molecules', ascending=False)
tmp.index.name = 'Archive'
tmp.head()

,# Molecules,Behavior,Physics
Archive,,,
foodb,65819,False,True
knapsack,48818,False,False
flavordb,25595,True,False
sharma_2021b,5105,True,True
goodscents,4565,True,False


In [43]:
all_molecules = {}
for arc in tmp[tmp.Behavor == True].index.to_list():
    if arc != 'flavordb':
        try:
            all_molecules[arc] = pyrfume.load_data(f'{arc}/molecules.csv')
        except:
            print(f'No molecules.csv found for {arc}')

No molecules.csv found for nhanes_2014


In [4]:
all_molecules = {}
for arc in archives:
    try:
        all_molecules[arc] = pyrfume.load_data(f'{arc}/molecules.csv')
    except:
        print(f'No molecules.csv found for {arc}')

No molecules.csv found for nhanes_2014


In [44]:
df = pd.concat(all_molecules, axis=0).reset_index().rename(columns={'level_0': 'Archive'})
df = df.set_index('CID').sort_index().drop_duplicates()
df = df[['MolecularWeight', 'IsomericSMILES', 'IUPACName', 'name', 'Archive']]

print(df.shape)
df.head()

(28247, 5)


,MolecularWeight,IsomericSMILES,IUPACName,name,Archive
CID,,,,,
-35,188.336,CC(C)CC(C)(O)C1CCCS1,NaN,Isopropylmethyltetrahydrothiophenyl-ethanol,leffingwell
-34,300.395,CCCCCCCCC(=O)C(=O)OC(=C(C)C)C(O)CO,NaN,Isopropylideneglyceryl oxodecanoate,leffingwell
-33,224.344,CCCCC=COC(=O)C=CCCCCC,NaN,Hexenyl octenoate,leffingwell
-32,222.350,CCCC(CCOC)[SH]=C(O)OCC,NaN,Ethyl S-(1-methoxyhexan-3-yl)carbonothioate,leffingwell
-31,188.336,CCCCCCCCC(S)C=O,NaN,Mercaptodecanal,leffingwell


In [138]:
# df.to_csv(os.path.join('..', 'static', 'molecule_master_list.csv'))